# Лабораторная работа No3
## Задача
1. Возьмите данные, представленные в файле
`Data/Lab3/Steam_2024_bestRevenue_1500.csv`:

    Загрузите данные в вашу схему данных.  
    Сделайте нормализацию модели данных, выделив в отдельные таблицы следующие атрибуты из файла:
    - publishers (название компании, кто опубликовал игру)
    - publisherClass (класс игры)
    - developers (команда разработчиков игры)

    Полученные 4 таблицы должны быть связаны внешними ключами.

## Создаем подключение к БД

In [37]:
import csv
import psycopg2
from datetime import datetime
from tabulate import tabulate

### Подключение к PostgreSQL
connection = psycopg2.connect(
    host="localhost",
    port="5432",
    database="",
    user="",
    password="",
)
### Создание курсора
cursor = connection.cursor()

print("Подключение к PostgreSQL успешно.")

Подключение к PostgreSQL успешно.


## Создание схемы

In [38]:
cursor.execute("CREATE SCHEMA IF NOT EXISTS ivan_patakin;")
connection.commit()

print("Схема 'ivan_patakin' создана.")

Схема 'ivan_patakin' создана.


## Создание таблиц

In [39]:
cursor.execute("""
    CREATE TABLE ivan_patakin.publishers (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) UNIQUE NOT NULL
    );
""")
connection.commit()

cursor.execute("""
    CREATE TABLE ivan_patakin.publisher_class (
        id SERIAL PRIMARY KEY,
        class VARCHAR(255) UNIQUE NOT NULL
    );
""")
connection.commit()

cursor.execute("""
    CREATE TABLE ivan_patakin.developers (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) UNIQUE NOT NULL
    );
""")
connection.commit()

cursor.execute("""
    CREATE TABLE ivan_patakin.games (
        steam_id BIGINT PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        release_date DATE,
        copies_sold INT,
        price NUMERIC(10, 2),
        revenue NUMERIC(15, 2),
        avg_playtime NUMERIC(10, 2),
        review_score INT,
        publisher_ids INT[] DEFAULT ARRAY[]::INT[],
        class_id INT REFERENCES ivan_patakin.publisher_class(id),
        developer_ids INT[] DEFAULT ARRAY[]::INT[]
    );
""")
connection.commit()

print("Таблицы созданы.")

Таблицы созданы.


## Загрузка данных из CSV

### Наполняем таблицы
- publishers
- publisher_class
- developers

In [40]:
unique_publishers = set()
unique_classes = set()
unique_developers = set()
games_data = []

with open('Data/Lab3/Steam_2024_bestRevenue_1500.csv', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Разделяем значения publishers и developers по запятой
        if row['publishers']:
            publishers = [pub.strip() for pub in row['publishers'].split(',')]
            unique_publishers.update(publishers)
        if row['publisherClass']:
            unique_classes.add(row['publisherClass'])
        if row['developers']:
            developers = [dev.strip() for dev in row['developers'].split(',')]
            unique_developers.update(developers)
        games_data.append(row)

# Вставляем уникальные данные в publishers
for publisher in unique_publishers:
    cursor.execute("""
    INSERT INTO ivan_patakin.publishers (name)
    VALUES (%s) ON CONFLICT (name) DO NOTHING;
    """, (publisher,))

connection.commit()
print("Таблица publishers заполнена")

# Вставляем уникальные данные в publisher_class
for publisher_class in unique_classes:
    cursor.execute("""
    INSERT INTO ivan_patakin.publisher_class (class)
    VALUES (%s) ON CONFLICT (class) DO NOTHING;
    """, (publisher_class,))

connection.commit()
print("Таблица publisher_class заполнена")

# Вставляем уникальные данные в developers
for developer in unique_developers:
    cursor.execute("""
    INSERT INTO ivan_patakin.developers (name)
    VALUES (%s) ON CONFLICT (name) DO NOTHING;
    """, (developer,))

connection.commit()
print("Таблица developers заполнена")

Таблица publishers заполнена
Таблица publisher_class заполнена
Таблица developers заполнена


### Теперь наполняем таблицу games

In [41]:
for row in games_data:
    # Получаем массив ID для publishers
    publisher_ids = []
    if row['publishers']:
        publishers = [pub.strip() for pub in row['publishers'].split(',')]
        for publisher in publishers:
            cursor.execute("SELECT id FROM ivan_patakin.publishers WHERE name = %s;", (publisher,))
            result = cursor.fetchone()
            if result:
                publisher_ids.append(result[0])

    # Получаем массив ID для developers
    developer_ids = []
    if row['developers']:
        developers = [dev.strip() for dev in row['developers'].split(',')]
        for developer in developers:
            cursor.execute("SELECT id FROM ivan_patakin.developers WHERE name = %s;", (developer,))
            result = cursor.fetchone()
            if result:
                developer_ids.append(result[0])

    # Получаем ID для class_id
    class_id = None
    if row['publisherClass']:
        cursor.execute("SELECT id FROM ivan_patakin.publisher_class WHERE class = %s;", (row['publisherClass'],))
        result = cursor.fetchone()
        if result:
            class_id = result[0]

    # Вставляем данные в games
    cursor.execute("""
    INSERT INTO ivan_patakin.games (
        steam_id, title, release_date, copies_sold, price, revenue, avg_playtime, review_score,
        publisher_ids, class_id, developer_ids
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (
        int(row['steamId']),
        row['name'],
        datetime.strptime(row['releaseDate'], '%d-%m-%Y').date(),
        int(row['copiesSold']),
        float(row['price']) if row['price'] else None,
        float(row['revenue']),
        float(row['avgPlaytime']),
        int(row['reviewScore']) if row['reviewScore'] else None,
        publisher_ids,
        class_id,
        developer_ids
    ))

connection.commit()

print("Данные успешно загружены.")

Данные успешно загружены.


## Проверка данных

In [42]:
print("Таблица publishers")
cursor.execute("""
    SELECT name FROM ivan_patakin.publishers
""")

rows = cursor.fetchall()
headers = ["Название"]
print(tabulate(rows, headers=headers))

Таблица publishers
Название
-------------------------------------------
HeadArrow
Akupara Games
Gamersky Games
Denpasoft
Bandai Namco Entertainment America Inc.
Shiny Shoe
CM Games
战国女神
Euphoric Brothers
MicroProse Software
Games Incubator
Inceton games
Childish Things Ltd
Cozy Cabin Studios
Toyasky
方块游戏(CubeGame)
Tora Creatives
Robot Assembly
零创游戏(ZerocreationGame)
SUNJOY GAME
Inflexion Games
INSTINCT3
Good Shepherd Entertainment
Thunderoak Interactive
Apogee Entertainment
Mash Games
HFTGames
OMEGAMES STUDIO
Topgame
PLAION
Grey Alien Games
Hunter Studio
Concrete Interactive顽石发行
FiftyEight Studios
Kalypso Media
Taboo Tales 💘
No Serious Games
Deniz K.
Rosa Special Studio
ArcaneSpoon
Beawesome Games LLC
Stumbling Cat
Hotloop
Bethesda Softworks
Curve Games
Star Drifters
Brave Cat Games
Weakfish Studio Publishing
NewSmile Games
HappyFairy213
DuaWolf Studio
Mega Cat Studios
Mayni prod.
SUNSOFT
One Hamsa
Coffee Stain Publishing
Kairosoft Co.
Bloodbath Studios
Hematite
Thermite Games
DeadDevs

In [43]:
print("Таблица publisher_class")
cursor.execute("""
    SELECT class FROM ivan_patakin.publisher_class
""")

rows = cursor.fetchall()
headers = ["Название"]
print(tabulate(rows, headers=headers))

Таблица publisher_class
Название
----------
AAA
AA
Indie
Hobbyist


In [44]:
print("Таблица developers")
cursor.execute("""
    SELECT name FROM ivan_patakin.developers
""")

rows = cursor.fetchall()
headers = ["Название"]
print(tabulate(rows, headers=headers))

Таблица developers
Название
----------------------------------------
Mauris
Games Incubator
Cozy Cabin Studios
Inceton games
Still Running
ShiroKuroSoft
Topgame
Grey Alien Games
黃金
Super Trampers Studio
Mega Cat Studios
One Hamsa
Salamander Interactive
Kairosoft Co.
La Moutarde
MONKEYCRAFT Co. Ltd.
Ghostlight LTD
HOGO Games
Chinesegamer
Best Way
SinisterGames
Borington
Never Games Limited
CLLGames
@unepic_fran
Supergiant Games
Disaster Studio
Awaceb
14 Hours Productions
Weeping studio
Feperd Games
Argonwood
SweetRaspberry
老乌贼工作室
Love Seekers
HAMMER95
WSS playground
cockydoody
Thigh High Games
MIGI STUDIO
Pixellore Inc
Cat Paw Game
Evil Boobs Cult
Robin Nicolet
iolaCorp Studio
Longplay Studios
Monkey Craft
ofuroworks
GameFormatic S.A.
Skog
Pixel Ferrets
Thunderful Development
CodeParade
AIKUN
Good-Feel Co.
Squid Shock Studios
Christopher Stair
Shuuko
ankaa studio
Friedemann
Miju Games
Fushidaratei
Lizzycat / Lizabeth Gidlund
Robo Cat
Akatsuki Games Inc.
Illwinter Game Design
René Rother

In [45]:
print("Таблица games")
cursor.execute("""
    SELECT * FROM ivan_patakin.games ORDER BY steam_id;
""")

rows = cursor.fetchall()
headers = ["steam_id", "Название", "Дата релиза", "Проданные копии", "Цена", "Доход", "Среднее время игры", "Оценка", "publisher_ids", "class_id", "developer_ids"]
print(tabulate(rows, headers=headers))

Таблица games
  steam_id  Название                                                                                                                                                                                                         Дата релиза      Проданные копии    Цена             Доход    Среднее время игры    Оценка  publisher_ids                class_id  developer_ids
----------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -------------  -----------------  ------  ----------------  --------------------  --------  -------------------------  ----------  -------------------------------
     24880  The Saboteur™                                                                                                                                                                                                    2024-03-07     

## Создадим view для таблицы game

In [46]:
cursor.execute("""
    CREATE VIEW ivan_patakin.games_view AS
    SELECT 
        g.steam_id,
        g.title AS game_title,
        COALESCE(g.release_date::TEXT, '-') AS release_date,
        COALESCE(TO_CHAR(g.copies_sold, 'FM999,999,999.00'), '-') AS copies_sold,
        COALESCE(TO_CHAR(g.price, 'FM999,999,999.00'), '-') AS price,
        COALESCE(TO_CHAR(g.revenue, 'FM999,999,999.00'), '-') AS revenue,
        COALESCE(TO_CHAR(g.avg_playtime, 'FM999,999,999.00'), '-') AS avg_playtime,
        COALESCE(g.review_score::TEXT, '-') AS review_score,
        COALESCE(STRING_AGG(DISTINCT p.name, ', '), '-') AS publisher_names,
        COALESCE(pc.class, '-') AS publisher_class,
        COALESCE(STRING_AGG(DISTINCT d.name, ', '), '-') AS developer_names
    FROM 
        ivan_patakin.games g
    LEFT JOIN 
        ivan_patakin.publisher_class pc ON g.class_id = pc.id
    LEFT JOIN 
        LATERAL UNNEST(g.publisher_ids) AS pub_id ON TRUE 
    LEFT JOIN 
        ivan_patakin.publishers p ON pub_id = p.id
    LEFT JOIN 
        LATERAL UNNEST(g.developer_ids) AS dev_id ON TRUE
    LEFT JOIN 
        ivan_patakin.developers d ON dev_id = d.id
    GROUP BY 
        g.steam_id, g.title, g.release_date, g.copies_sold, g.price, g.revenue, 
        g.avg_playtime, g.review_score, pc.class;
""")
connection.commit()
print("Создано View для games.")

Создано View для games.


In [47]:
cursor.execute(""" SELECT * FROM ivan_patakin.games_view ORDER BY steam_id; """)

rows = cursor.fetchall()
headers = ["steam_id", "Название", "Дата релиза", "Проданные копии", "Цена", "Доход", "Среднее время игры", "Оценка", "Издатель", "Класс", "Разработчик"]

print(tabulate(
    rows,
    headers=headers,
    tablefmt="grid",
    colalign=("right", "left", "right", "right", "right", "right", "right", "right", "left", "left", "left"),
    maxcolwidths=[20, 20, 15, 20, 10, 20, 15, 10, 20, 10, 20]
))

+------------+----------------------+---------------+-------------------+--------+----------------+----------------------+----------+----------------------+----------+----------------------+
|   steam_id | Название             |   Дата релиза |   Проданные копии |   Цена |          Доход |   Среднее время игры |   Оценка | Издатель             | Класс    | Разработчик          |
+============+======================+===============+===================+========+================+======================+==========+======================+==========+======================+
|      24880 | The Saboteur™        |    2024-03-07 |         53,376.00 |   9.99 |     290,153.00 |                12.83 |       92 | Electronic Arts      | AAA      | Pandemic Studios     |
+------------+----------------------+---------------+-------------------+--------+----------------+----------------------+----------+----------------------+----------+----------------------+
|     251570 | 7 Days to Die        |    2024

## Удаление таблиц

In [48]:
cursor.execute("""
    DROP VIEW IF EXISTS  ivan_patakin.games_view;
    DROP TABLE IF EXISTS ivan_patakin.games;
    DROP TABLE IF EXISTS ivan_patakin.developers;
    DROP TABLE IF EXISTS ivan_patakin.publisher_class;
    DROP TABLE IF EXISTS ivan_patakin.publishers;
""")

connection.commit()

print("Таблицы удалены")

Таблицы удалены


### Закрытие курсора и соединения

In [49]:
cursor.close()
connection.close()

print("Соединение с PostgreSQL закрыто.")

Соединение с PostgreSQL закрыто.
